In [41]:
# !pip install pyvi

In [4]:
import pandas as pd
import os
import sys
from tqdm import tqdm
from pyvi import ViTokenizer

sys.path.append('../src/dtprocess')
import cleandt

In [28]:
# from google.colab import drive
# drive.mount('/content/drive')

In [29]:
# %cd /content/drive/My Drive/Colab Notebooks/Text_Generator

In [30]:
# %ls -la

## Transform the raw data to usable text

* Load raw data from source

In [7]:
CRAWL_FOLDER = '../data/vnexpress/raw_news'
vnexpress = []

for filename in os.listdir(CRAWL_FOLDER):
    with open('../data/vnexpress/raw_news/{}'.format(filename), 'r') as file:
        news = file.readlines()
        vnexpress += cleandt.convert_dict(news, 'content')

In [8]:
news = pd.DataFrame(columns=['content','url','topic', 'sub-topic', 'image', 'title','description'])

for new in tqdm(vnexpress):
    news.loc[len(news)] = pd.Series(new)
news = news.reset_index().rename(columns={'index':'article_id'})

In [9]:
processed_news = news[['article_id','content','topic','sub-topic','title','description']]
print(processed_news.shape)
print(processed_news.info())

(2481, 6)


In [10]:
processed_news.isna().sum()

article_id     0
content        0
topic          0
sub-topic      0
title          2
description    7
dtype: int64

In [11]:
processed_news.dropna(axis=0, inplace=True)

C:\Users\buidu\AppData\Local\Temp\ipykernel_12544\1992709397.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_news.dropna(axis=0, inplace=True)


In [12]:
processed_news['tag'] = processed_news['content'] + processed_news['title'] + processed_news['description']
processed_news = processed_news.drop(columns=['content','description','title'])

C:\Users\buidu\AppData\Local\Temp\ipykernel_12544\3994754769.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_news['tag'] = processed_news['content'] + processed_news['title'] + processed_news['description']


In [13]:
processed_news['tag'] = processed_news['tag'].apply(lambda x: x.lower())
processed_news['tag'] = processed_news['tag'].apply(lambda x: cleandt.remove_punctuation(x))
processed_news['tag'] = processed_news['tag'].apply(lambda x: ViTokenizer.tokenize(x))

In [24]:
print(processed_news['topic'].value_counts())
processed_news.head()

topic
giai-tri    616
the-thao    508
khoa-hoc    449
doi-song    363
giao-duc    358
du-lich     180
Name: count, dtype: int64


,article_id,topic,sub-topic,tag
0,0,doi-song,to-am,xin_lỗi đi nhiều thế_hệ đã dùng câu này để xử_...
1,1,doi-song,to-am,một nghiên_cứu của đại_học wisconsinmadison mỹ...
2,2,doi-song,to-am,các yếu_tố như độ tuổi của trẻ mối quan_hệ giữ...
3,3,doi-song,to-am,ở tuổi đôi_mươi chị trần thị liên_kết_hôn với ...
4,4,doi-song,to-am,chị hoàng thị hòa 34 tuổi ở hà nam lấy chồng b...


## Dump each tag to a text file

In [18]:
PROCESSED_FOLDER = '../data/vnexpress/processed_news'

In [19]:
os.listdir(PROCESSED_FOLDER)

['doi-song', 'du-lich', 'giai-tri', 'giao-duc', 'khoa-hoc', 'the-thao']

In [25]:
def get_info(topic):
  temp = processed_news[processed_news.topic == topic]
  return temp['article_id'].to_list(), temp['tag'].to_list()

In [26]:
for topic in tqdm(os.listdir(PROCESSED_FOLDER)):
  articles_ids, tags = get_info(topic)
  # print(articles_ids)
  for id, tag in zip(articles_ids, tags):
    with open(f'{PROCESSED_FOLDER}/{topic}/{id}.txt', "w", encoding="utf-8") as file:
      file.write(tag)

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:02<00:00,  2.23it/s]


In [27]:
processed_news.to_csv('../data/vnexpress/csv/cleaned_vnexpress.csv')